# Here I add the materials science specific functions

Terminology:
- SIC: symmetry independent configuration
- SEC: symmetry equivalent configuration

In [104]:
from pymatgen.core.structure import Structure, Molecule
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer, PointGroupAnalyzer
import copy
import numpy as np
from ase.visualize import view
from pymatgen.io.ase import AseAtomsAdaptor
import itertools

def vview(structure):
    view(AseAtomsAdaptor().get_atoms(structure))

## Build the reference 2x2 graphene supercell

Build a supercell with pymatgen

#### PBC

In [168]:
lattice = np.array([[ 1.233862, -2.137112,  0.      ],
                   [ 1.233862,  2.137112,  0.      ],
                   [ 0.      ,  0.      ,  8.685038]])

graphene = Structure(lattice, species=['C','C'], coords=[[2/3, 1/3, 0. ],[1/3, 2/3, 0.]])
graphene = SpacegroupAnalyzer(graphene).get_conventional_standard_structure()

supercell_order = 2
scaling_matrix = np.array([[supercell_order, 0, 0],
                            [0, supercell_order, 0],
                            [0, 0, 1]])
graphene_scell = copy.deepcopy(graphene)
graphene_scell.make_supercell(scaling_matrix)
# Reorder the atoms in the supercell so they follow the convention we are using (top to bottom, left to right).
ordering = [1,5,0,3,4,7,2,6]
graphene_scell = Structure(graphene_scell.lattice,graphene_scell.atomic_numbers,
                           graphene_scell.frac_coords[ordering])


#### No PBC

In [171]:
lattice = np.array([[ 1.233862, -2.137112,  0.      ],
                   [ 1.233862,  2.137112,  0.      ],
                   [ 0.      ,  0.      ,  8.685038]])

graphene = Structure(lattice, species=['C','C'], coords=[[2/3, 1/3, 0. ],[1/3, 2/3, 0.]])
graphene = SpacegroupAnalyzer(graphene).get_conventional_standard_structure()

supercell_order = 2
scaling_matrix = np.array([[supercell_order, 0, 0],
                            [0, supercell_order, 0],
                            [0, 0, 1]])
graphene_scell = copy.deepcopy(graphene)
graphene_scell.make_supercell(scaling_matrix)
# Reorder the atoms in the supercell so they follow the convention we are using (top to bottom, left to right).
ordering = [1,5,0,3,4,7,2,6]
graphene_scell = Molecule(graphene_scell.atomic_numbers,
                           graphene_scell.cart_coords[ordering])


## Symmetry analysis

In [246]:
def get_all_configurations_pbc(structure):
    
    # structure: pymatgen Structure object

    symmops = SpacegroupAnalyzer(structure).get_symmetry_operations()
    
    #print(symmop)
    coordinates = np.array(structure.frac_coords)
    n_symmops = len(symmops)
    atom_numbers = np.array(structure.atomic_numbers)
    lattice = structure.lattice.matrix
    
    original_structure = copy.deepcopy(structure)
            
    atom_indices = []
    structures = []
    for i,symmop in enumerate(symmops[0:10]):
        atom_indices_tmp = []
        coordinates_new = []
        for site in coordinates:
            coordinates_new.append(symmop.operate(site))

        structure_tmp = Structure(lattice,atom_numbers,coordinates_new,coords_are_cartesian=False,to_unit_cell=False)
        vview(structure_tmp)
        for k,coord in enumerate(original_structure.frac_coords):
            structure_tmp.append(original_structure.atomic_numbers[k],coord,coords_are_cartesian=False,validate_proximity=False)
        
        for m in range(len(atom_numbers)):
            index = len(atom_numbers)+m
            for n in range(len(atom_numbers)):
                #print(m,n,structure_tmp.frac_coords[n]-structure_tmp.frac_coords[index],structure_tmp.sites[n].is_periodic_image(structure_tmp.sites[index]))
                if structure_tmp.sites[n].is_periodic_image(structure_tmp.sites[index]):
                    atom_indices_tmp.append(n)
                    break
        atom_indices.append(atom_indices_tmp)

    return atom_indices
#atom_indices = get_all_configurations_pbc(graphene_scell)

AttributeError: 'Molecule' object has no attribute 'lattice'

In [238]:
def get_all_configurations_no_pbc(structure):
    
    # structure: pymatgen Molecule object
    
    symmops = PointGroupAnalyzer(structure).get_symmetry_operations()
    structure.translate_sites(np.arange(structure.num_sites),-structure.center_of_mass)

    coordinates = np.array(structure.cart_coords)
    n_symmops = len(symmops)
    atom_numbers = np.array(structure.atomic_numbers)
    
    original_structure = copy.deepcopy(structure)
            
    atom_indices = []
    structures = []
    for i,symmop in enumerate(symmops):
        atom_indices_tmp = []
        coordinates_new = []
        for site in coordinates:
            coordinates_new.append(symmop.operate(site))

        structure_tmp = Molecule(atom_numbers,coordinates_new)
        structure_tmp.translate_sites(np.arange(structure_tmp.num_sites),-structure_tmp.center_of_mass)

        for k,coord in enumerate(original_structure.cart_coords):
            structure_tmp.append(original_structure.atomic_numbers[k],coord,validate_proximity=False)
        #print(np.unique(np.round(structure_tmp.cart_coords,4),axis=0))
        #vview(structure_tmp)
        for m in range(len(atom_numbers)):
            index = len(atom_numbers)+m
            for n in range(len(atom_numbers)):
                #print(m,n,structure_tmp.frac_coords[n]-structure_tmp.frac_coords[index],structure_tmp.sites[n].is_periodic_image(structure_tmp.sites[index]))
                #if structure_tmp.sites[n].is_periodic_image(structure_tmp.sites[index]):
                if np.all(np.round(structure_tmp.cart_coords[n],4) == np.round(structure_tmp.cart_coords[index],4)):
                    atom_indices_tmp.append(n)
                    break
        atom_indices.append(atom_indices_tmp)

    return atom_indices
#atom_indices = get_all_configurations_no_pbc(graphene_scell)

In [239]:
def build_symmetry_equivalent_configurations(atom_indices,N_index):
    
    if len(N_index) == 0:

        return np.tile(np.zeros(len(atom_indices[0]),dtype='int'), (len(atom_indices), 1))
    configurations = atom_indices == -1
    for index in N_index:
        configurations += atom_indices == index
    configurations = configurations.astype(int)

    unique_configurations,unique_configurations_index = np.unique(configurations,axis=0,return_index=True)
    
    return unique_configurations


In [240]:
def find_sic(configurations,energies,atom_indices):
    
    # configurations : 2D array where each line is a binary vector
    # energies: list of energies (one per configuration)
    # atom_indices: atom indices returned by get_all_configurations_no_pbc or get_all_configurations_pbc
    
    config_unique = []
    multiplicity = []
    keep_energy = [] 
    for i,config in enumerate(configurations):
        
        sites = np.where(config == 1)[0] 
        sec = build_symmetry_equivalent_configurations(atom_indices,sites)
        sic = sec[0]
        is_in_config_unique = any(np.array_equal(sic, existing_sic) for existing_sic in config_unique)
        
        if not is_in_config_unique:  

            config_unique.append(sic)

            multiplicity.append(len(sec))
            keep_energy.append(i)
    unique_energies = np.array(energies)[keep_energy]
    
    return config_unique, unique_energies, multiplicity
    

Example on how to use the above. 

In this case the energies are random numbers, but in a real scenario they would be the same for SIC.

In [248]:
#Generate all configurations (you would use yours)
configurations = np.array(list(itertools.product([0, 1], repeat=graphene_scell.num_sites)))
#Generate random energies (you would use yours)
energies = np.random.rand(len(configurations))

atom_indices = get_all_configurations_no_pbc(graphene_scell)

config_unique, unique_energies, multiplicity = find_sic(configurations,energies,atom_indices)

## Periodic boudnary conditions